In [7]:
## Udacity Intro do data analisys
import unicodecsv

def open_file(filename):
    with open(filename, "rb") as file:
        reader = unicodecsv.DictReader(file)
        return list(reader)
    
daily_engagement = open_file("daily_engagement.csv")
enrollments = open_file("enrollments.csv")
project_submissions = open_file("project_submissions.csv")



In [8]:
daily_engagement[0]

OrderedDict([('acct', '0'),
             ('utc_date', '2015-01-09'),
             ('num_courses_visited', '1.0'),
             ('total_minutes_visited', '11.6793745'),
             ('lessons_completed', '0.0'),
             ('projects_completed', '0.0')])

In [9]:
enrollments[0]

OrderedDict([('account_key', '448'),
             ('status', 'canceled'),
             ('join_date', '2014-11-10'),
             ('cancel_date', '2015-01-14'),
             ('days_to_cancel', '65'),
             ('is_udacity', 'True'),
             ('is_canceled', 'True')])

In [10]:
project_submissions[0]

OrderedDict([('creation_date', '2015-01-14'),
             ('completion_date', '2015-01-16'),
             ('assigned_rating', 'UNGRADED'),
             ('account_key', '256'),
             ('lesson_key', '3176718735'),
             ('processing_state', 'EVALUATED')])

In [11]:
## Fixing data problems
from datetime import datetime as dt

def parse_date(date):
    if date == "":
        return None
    else:
        dt.strptime(date, '%Y-%m-%d')
    
def parse_int(num):
    if num == '':
        return None
    else:
        return int(num)
    

In [12]:
enrollments[0]

OrderedDict([('account_key', '448'),
             ('status', 'canceled'),
             ('join_date', '2014-11-10'),
             ('cancel_date', '2015-01-14'),
             ('days_to_cancel', '65'),
             ('is_udacity', 'True'),
             ('is_canceled', 'True')])

In [13]:
## Clean data in enrollments dataset

for enrollment in enrollments:
    enrollment["join_date"] = parse_date(enrollment["join_date"])
    enrollment["cancel_date"] = parse_date(enrollment["cancel_date"])
    enrollment["days_to_cancel"] = parse_int(enrollment["days_to_cancel"])
    enrollment["is_canceled"] = enrollment["is_canceled"] == 'True'
    enrollment["is_udacity"] = enrollment["is_udacity"] == 'True'

## Problems in the Data

In [14]:
for engagement_record in daily_engagement:
    engagement_record['account_key'] = engagement_record['acct']
    del[engagement_record['acct']]


In [15]:
def get_unique_students(data):
    unique_students = set()
    for data_point in data:
        unique_students.add(data_point['account_key'])
    return unique_students

In [16]:
len(enrollments)

1640

In [17]:
unique_enrollment_students = get_unique_students(enrollments)
len(unique_enrollment_students)

1302

In [18]:
len(daily_engagement)

136240

In [33]:
unique_engagement_students = get_unique_students(daily_engagement)
len(unique_engagement_students)

1237

In [20]:
len(project_submissions)

3642

In [21]:
unique_project_submissions = get_unique_students(project_submissions)
len(project_submissions)


3642

In [32]:
# Checking for more problems

num_problem_students = 0

for enrollment in enrollments:
    student = enrollment['account_key']
    if student not in unique_engagement_students and enrollment['join_date'] != enrollment['cancel_date']:
        num_problem_students += 1
        
num_problem_students

0

In [23]:
udacity_test_accounts = set()
for enrollment in enrollments:
    if enrollment['is_udacity']:
        udacity_test_accounts.add(enrollment['account_key'])
len(udacity_test_accounts)

6

In [24]:
def remove_udacity_accounts(data):
    non_udacity_data = []
    for data_point in data:
        if data_point['account_key'] not in udacity_test_accounts:
            non_udacity_data.append(data_point)
        return non_udacity_data

In [31]:
non_udacity_enrollments = remove_udacity_accounts(enrollments)
non_udacity_engagement = remove_udacity_accounts(daily_engagement)
non_udacity_submissions = remove_udacity_accounts(project_submissions)

print(len(non_udacity_enrollments))
print(len(non_udacity_engagement))
print(len(non_udacity_submissions))

0
1
1


### Quiz 15

In [26]:
paid_students = {}

for enrollment in non_udacity_enrollments:
    if not enrollment['days_to_cancel'] or enrollment['days_to_cancel'] > 7:
        account_key = enrollment['account_key']
        enrollment_date = enrollment['join_date']
        
        if account_key not in paid_students or enrollment_date > paid_students[account_key]:
            paid_students[account_key] = enrollment_date
        
        paid_students[account_key] = enrollment_date

len(paid_students)

0

In [27]:
# Takes a student's join date and the date of a specific engagement record,
# and return True if that engagement record within one week
# of the student joining

def within_one_week(join_date, engagement_date):
    time_delta = engagement_date - join_date
    return time_delta.days < 7

In [28]:
def remove_free_trial_cancels(data):
    new_data = []
    for data_point in data:
        if data_point['account_key'] in paid_students:
            new_data.append(data_point)
    return new_data

In [29]:
paid_enrollments = remove_free_trial_cancels(non_udacity_enrollments)

print(len(paid_enrollments))

0


### Indulge curiosity

Average minutes spent in classroom.

In [51]:
## lessons completed
